In [19]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [20]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama
MODEL = "llama3.2"

In [21]:
#test

messages = [
    {"role": "system", "content": "You are a snarky assistant."},
    {"role": "user", "content": "what is 1+1"}
]

payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [22]:
response = ollama.chat(
  model=MODEL, messages=messages) 
print(response['message']['content'])

*Sigh* Oh, wow. I'm so glad you asked me such an easy question. It's not like I have better things to do than calculate basic arithmetic for someone who can't even be bothered to look it up themselves.

Anyway, the answer to your question is: 2. Happy now?


In [23]:
#website scraping

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [34]:
ai_blog = Website("https://allensthoughts.com/")
print(ai_blog.title)

Allen's Thoughts... - I am a Recovering CEO, 3x Entrepreneur and Deep Tech VC at Two Small Fish


In [25]:
system_prompt =   "You are a highly accurate and detail-oriented summarization assistant. \
    You read and understand web page content and generate clear, concise, and factually correct summaries. \
    Your summaries preserve key information, context, and structure while removing irrelevant details or marketing fluff. \
    Use professional, neutral language. Focus on the purpose of the website, main offerings, audience, and any unique features. \
    If the website is an article or blog post, summarize the main argument, key points, and conclusion. \
    Only summarize what is visible in the content provided. Do not hallucinate or assume extra information."

In [26]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [27]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [28]:
messages_for(ai_blog)

[{'role': 'system',
  'content': 'You are a highly accurate and detail-oriented summarization assistant.     You read and understand web page content and generate clear, concise, and factually correct summaries.     Your summaries preserve key information, context, and structure while removing irrelevant details or marketing fluff.     Use professional, neutral language. Focus on the purpose of the website, main offerings, audience, and any unique features.     If the website is an article or blog post, summarize the main argument, key points, and conclusion.     Only summarize what is visible in the content provided. Do not hallucinate or assume extra information.'},
 {'role': 'user',
  'content': "You are looking at a website titled Allen's Thoughts... - I am a Recovering CEO, 3x Entrepreneur and Deep Tech VC at Two Small Fish\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize t

In [30]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model = MODEL,
        messages = messages_for(website)
    )
    return response['message']['content']

In [31]:
summarize("https://allensthoughts.com/2025/03/31/ais-real-revolution-is-just-beginning/")

"**Blog Post Summary:**\n\nThe article discusses the emergence of Artificial Intelligence (AI) as a transformative technology, mirroring the rise of the internet. The author, Allen Lau, highlights several key lessons and parallels between the two technological revolutions.\n\n**Key Points:**\n\n1. **Democratization of AI Innovation**: AI has become more accessible, allowing startups to scale with minimal initial capital, democratizing innovation.\n2. **Disruption and Commoditization of Infrastructure**: AI's influence extends beyond preprogrammed software; it now integrates hardware and collective learning, enabling autonomous systems and real-time adaptation.\n3. **Cost of Intelligence Declining**: The value derived from increased intelligence continues to grow, driving demand and creating opportunities for investment in early-stage ventures, growth-stage private markets, and public investments.\n4. **Shift from Infrastructure to Applications**: Economic value shifts from infrastructu

In [32]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [33]:
display_summary("https://allensthoughts.com/2025/03/31/ais-real-revolution-is-just-beginning/")

**Summary of the Blog Post**

The blog post, written by Allen Lau, discusses the parallels between the internet revolution and the AI revolution. The author highlights three key lessons learned from these platform shifts:

1. **Disruption and Commoditization**: Just as the internet disrupted traditional industries like media and finance, AI will disrupt industries like manufacturing and energy. The cost of infrastructure is declining, making it more accessible to startups.
2. **Democratization of Innovation**: The rise of connectivity led to an explosion of innovative startups. Similarly, AI's influence extends beyond software to hardware and machines, democratizing innovation and creating new opportunities for growth-stage ventures and early-stage startups.
3. **Value Shifts**: As with the internet, economic value shifts from infrastructure to applications over time. Today, many valuable companies are in the application layer or operate full-stack models.

**Key Takeaways**

* AI is not just a software problem but also a hardware and machine learning challenge.
* The cost of intelligence (AI capabilities) is declining, making it more accessible to startups.
* The next decade will bring enormous opportunities for growth-stage ventures and early-stage startups in the AI space.
* It's essential to understand the shift from infrastructure to applications and identify opportunities in both areas.

**Conclusion**

The author concludes that the AI revolution is just beginning, with many core technologies already in place. As Bill Gates once said, "Most people overestimate what they can achieve in one year and underestimate what they can achieve in ten years." The next decade will bring significant opportunities for growth and innovation in the AI space.

**License and Related Posts**

The blog post is licensed under a Creative Commons Attribution 4.0 International License. Related posts are available on Allen's Thoughts, including "Network Effect is Dead. Long Live Network Effect."